<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/Dialogue_Speaker_Extract_Test_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日的对话抽取模型

我们希望有一个模型能够从小说的chunk中批量去提取摘要和对话

这个模型就是实现了这一点。模型使用了大约30k的中文小说数据和20k的英文小说数据进行训练，在qwen-1.8上进行了3个epoch的finetune。 原则上模型同时支持中文和英文小说的训练

主项目链接 https://github.com/LC1332/Chat-Haruhi-Suzumiya

- [李鲁鲁](https://github.com/LC1332)完成了数据的收集，以及进一步将inference程序扩展到连续的chunks
- [刘崇寒](https://github.com/khazic)完成了模型的训练
- [米唯实](https://github.com/hhhwmws0117)测试并上传模型到hugging face

# Chat Haruhi Suzumiya's Dialogue Extraction Model

We hope to have a model that can extract summaries and dialogues in batches from chunks of novels.

This model achieves just that. It was trained using approximately 30k Chinese novels and 20k English novels, and was fine-tuned on qwen-1.8 for three epochs. In principle, the model supports training for both Chinese and English novels.

Main project link: https://github.com/LC1332/Chat-Haruhi-Suzumiya

### TODO

- [x] 拓展到多chunks的inference
- [x] 提供英语的例子
- [ ] 提供一个多章节并行inference的例子
- [ ] 在json解析失败的时候尝试直接从raw字符串提取summary
- [ ] 在失败的时候额外尝试调用openai进行推理




### TODO

- [x] Expand to multi-chunk inference
- [x] Provide an English example
- [ ] Provide an example of multi-chapter parallel inference
- [ ] Try extracting summary directly from raw strings when JSON parsing fails
- [ ] Additionally attempt to use OpenAI for inference when failing


In [1]:
!pip install -q transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install -q auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is no

可能这里需要添加下HF_TOKEN

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract-csv", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract-csv", device_map="auto", trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract-csv:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/64.0 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract-csv:
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


cpp_kernels.py:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract-csv:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract-csv:
- modeling_qwen.py
- qwen_generation_utils.py
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

In [ ]:
# from transformers.generation import GenerationConfig

# model.generation_config = GenerationConfig.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18", trust_remote_code=True,top_k = 2, top_p = 0.9, max_new_tokens = 1000) # 可指定不同的生成长度、top_p等相关超参

In [5]:
print(model.generation_config)

GenerationConfig {
  "chat_format": "chatml",
  "do_sample": true,
  "eos_token_id": 151643,
  "max_new_tokens": 512,
  "max_window_size": 6144,
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "top_k": 0,
  "top_p": 0.8
}



In [6]:
#@title 定义raw_text

raw_text = """彭蠡家在小街深处。老刀上楼，彭蠡不在家。问邻居，邻居说他每天快到关门才回来，具体几点不清楚。

老刀有点担忧，看了看手表，清晨5点。

他回到楼门口等着。两旁狼吞虎咽的饥饿少年围绕着他。他认识其中两个，原来在彭蠡家见过一两次。少年每人面前摆着一盘炒面或炒粉，几个人分吃两个菜，盘子里一片狼藉，筷子扔在无望而锲而不舍地拨动，寻找辣椒丛中的肉星。老刀又下意识闻了闻小臂，不知道身上还有没有垃圾的腥味。周围的一切嘈杂而庸常，和每个清晨一样。

“哎，你们知道那儿一盘回锅肉多少钱吗？”那个叫小李的少年说。

“靠，菜里有沙子。”另外一个叫小丁的胖少年突然捂住嘴说，他的指甲里还带着黑泥， “坑人啊。得找老板退钱！”

“人家那儿一盘回锅肉，就三百四。”小李说，“三百四！一盘水煮牛肉四百二呢。”

“什么玩意？这么贵。”小丁捂着腮帮子咕哝道。

另外两个少年对谈话没兴趣，还在埋头吃面，小李低头看着他们，眼睛似乎穿过他们，看到了某个看不见的地方，目光里有热切。

老刀的肚子也感觉到饥饿。他迅速转开眼睛，可是来不及了，那种感觉迅速席卷了他，胃的空虚像是一个深渊，让他身体微微发颤。他有一个月不吃清晨这顿饭了。一顿饭差不多一百块，一个月三千块，攒上一年就够糖糖两个月的幼儿园开销了。

他向远处看，城市清理队的车辆已经缓缓开过来了。

他开始做准备，若彭蠡一时再不回来，他就要考虑自己行动了。虽然会带来不少困难，但时间不等人，总得走才行。身边卖大枣的女人高声叫卖，不时打断他的思绪，声音的洪亮刺得他头疼。步行街一端的小摊子开始收拾，人群像用棍子搅动的池塘里的鱼，倏一下散去。没人会在这时候和清理队较劲。小摊子收拾得比较慢，清理队的车耐心地移动。步行街通常只是步行街，但对清理队的车除外。谁若走得慢了，就被强行收拢起来。

这时彭蠡出现了。他剔着牙，敞着衬衫的扣子，不紧不慢地踱回来，不时打饱嗝。彭蠡六十多了，变得懒散不修边幅，两颊像沙皮狗一样耷拉着，让嘴角显得总是不满意地撇着。如果只看这幅模样，不知道他年轻时的样子，会以为他只是个胸无大志只知道吃喝的怂包。但从老刀很小的时候，他就听父亲讲过彭蠡的事。

老刀迎上前去。彭蠡看到他要打招呼，老刀却打断他：“我没时间和你解释。我需要去第一空间，你告诉我怎么走。”

彭蠡愣住了，已经有十年没人跟他提过第一空间的事，他的牙签捏在手里，不知不觉掰断了。他有片刻没回答，见老刀实在有点急了，才拽着他向楼里走。“回我家说，”彭蠡说，“要走也从那儿走。”

在他们身后，清理队已经缓缓开了过来，像秋风扫落叶一样将人们扫回家。“回家啦，回家啦。转换马上开始了。”车上有人吆喝着。

彭蠡带老刀上楼，进屋。他的单人小房子和一般公租屋无异，六平米房间，一个厕所，一个能做菜的角落，一张桌子一把椅子，胶囊床铺，胶囊下是抽拉式箱柜，可以放衣服物品。墙面上有水渍和鞋印，没做任何修饰，只是歪斜着贴了几个挂钩，挂着夹克和裤子。进屋后，彭蠡把墙上的衣服毛巾都取下来，塞到最靠边的抽屉里。转换的时候，什么都不能挂出来。老刀以前也住这样的单人公租房。一进屋，他就感到一股旧日的气息。

彭蠡直截了当地瞪着老刀：“你不告诉我为什么，我就不告诉你怎么走。”

已经5点半了，还有半个小时。

老刀简单讲了事情的始末。从他捡到纸条瓶子，到他偷偷躲入垃圾道，到他在第二空间接到的委托，再到他的行动。他没有时间描述太多，最好马上就走。

“你躲在垃圾道里？去第二空间？”彭蠡皱着眉，“那你得等24小时啊。”

“二十万块。”老刀说，“等一礼拜也值啊。”

“你就这么缺钱花？”

老刀沉默了一下。“糖糖还有一年多该去幼儿园了。”他说，“我来不及了。”

老刀去幼儿园咨询的时候，着实被吓到了。稍微好一点的幼儿园招生前两天，就有家长带着铺盖卷在幼儿园门口排队，两个家长轮着，一个吃喝拉撒，另一个坐在幼儿园门口等。就这么等上四十多个小时，还不一定能排进去。前面的名额早用钱买断了，只有最后剩下的寥寥几个名额分给苦熬排队的爹妈。这只是一般不错的幼儿园，更好一点的连排队都不行，从一开始就是钱买机会。老刀本来没什么奢望，可是自从糖糖一岁半之后，就特别喜欢音乐，每次在外面听见音乐，她就小脸放光，跟着扭动身子手舞足蹈。那个时候她特别好看。老刀对此毫无抵抗力，他就像被舞台上的灯光层层围绕着，只看到一片耀眼。无论付出什么代价，他都想送糖糖去一个能教音乐和跳舞的幼儿园。

彭蠡脱下外衣，一边洗脸，一边和老刀说话。说是洗脸，不过只是用水随便抹一抹。水马上就要停了，水流已经变得很小。彭蠡从墙上拽下一条脏兮兮的毛巾，随意蹭了蹭，又将毛巾塞进抽屉。他湿漉漉的头发显出油腻的光泽。

“你真是作死，”彭蠡说，“她又不是你闺女，犯得着吗。”

“别说这些了。快告我怎么走。”老刀说。

彭蠡叹了口气：“你可得知道，万一被抓着，可不只是罚款，得关上好几个月。”

“你不是去过好多次吗？”

“只有四次。第五次就被抓了。”

“那也够了。我要是能去四次，抓一次也无所谓。”
"""

In [10]:
# 定义get_token_len

text = "一句句子"

def get_token_len( text ):
    return len(tokenizer.encode(text))

get_token_len(text)

2

In [11]:
TOKEN_PER_CHUNK = 600

lines = raw_text.split("\n")

chunks = []
current_chunk = ""
current_len = 0

for line in lines:
    if line.strip() == "":
        continue
    line_len = get_token_len(line)
    if current_len + line_len > TOKEN_PER_CHUNK:
        if current_len > 0:
            chunks.append(current_chunk)
            current_chunk = line
            current_len = line_len

    else:
        current_chunk += line+ "\n"
        current_len += line_len

if current_len > 0:
    chunks.append(current_chunk)

print(len(chunks), " chunks")

for chunk in chunks:
    print(chunk)
    print('---')

3  chunks
彭蠡家在小街深处。老刀上楼，彭蠡不在家。问邻居，邻居说他每天快到关门才回来，具体几点不清楚。
老刀有点担忧，看了看手表，清晨5点。
他回到楼门口等着。两旁狼吞虎咽的饥饿少年围绕着他。他认识其中两个，原来在彭蠡家见过一两次。少年每人面前摆着一盘炒面或炒粉，几个人分吃两个菜，盘子里一片狼藉，筷子扔在无望而锲而不舍地拨动，寻找辣椒丛中的肉星。老刀又下意识闻了闻小臂，不知道身上还有没有垃圾的腥味。周围的一切嘈杂而庸常，和每个清晨一样。
“哎，你们知道那儿一盘回锅肉多少钱吗？”那个叫小李的少年说。
“靠，菜里有沙子。”另外一个叫小丁的胖少年突然捂住嘴说，他的指甲里还带着黑泥， “坑人啊。得找老板退钱！”
“人家那儿一盘回锅肉，就三百四。”小李说，“三百四！一盘水煮牛肉四百二呢。”
“什么玩意？这么贵。”小丁捂着腮帮子咕哝道。
另外两个少年对谈话没兴趣，还在埋头吃面，小李低头看着他们，眼睛似乎穿过他们，看到了某个看不见的地方，目光里有热切。
老刀的肚子也感觉到饥饿。他迅速转开眼睛，可是来不及了，那种感觉迅速席卷了他，胃的空虚像是一个深渊，让他身体微微发颤。他有一个月不吃清晨这顿饭了。一顿饭差不多一百块，一个月三千块，攒上一年就够糖糖两个月的幼儿园开销了。
他向远处看，城市清理队的车辆已经缓缓开过来了。
他开始做准备，若彭蠡一时再不回来，他就要考虑自己行动了。虽然会带来不少困难，但时间不等人，总得走才行。身边卖大枣的女人高声叫卖，不时打断他的思绪，声音的洪亮刺得他头疼。步行街一端的小摊子开始收拾，人群像用棍子搅动的池塘里的鱼，倏一下散去。没人会在这时候和清理队较劲。小摊子收拾得比较慢，清理队的车耐心地移动。步行街通常只是步行街，但对清理队的车除外。谁若走得慢了，就被强行收拢起来。

---
这时彭蠡出现了。他剔着牙，敞着衬衫的扣子，不紧不慢地踱回来，不时打饱嗝。彭蠡六十多了，变得懒散不修边幅，两颊像沙皮狗一样耷拉着，让嘴角显得总是不满意地撇着。如果只看这幅模样，不知道他年轻时的样子，会以为他只是个胸无大志只知道吃喝的怂包。但从老刀很小的时候，他就听父亲讲过彭蠡的事。老刀迎上前去。彭蠡看到他要打招呼，老刀却打断他：“我没时间和你解释。我需要去第一空间，你告诉我怎么走。”
彭蠡愣住了，已经有十年没人跟他提过第一空间的事，他的牙签捏在手里，不知不觉掰断了。他有片刻没

In [12]:
summary_prompt = """请对以下小说文本进行总结，原文:"""
extract_prompt = """给定input paragraph，以及相应的summary，抽取每一句对话的内容，判断每一句话的说话人 以<dialogue text> | said by | <speaker>的形式输出成csv格式"""

In [23]:
import json
summary = ""
model = model.eval()

dsp_text = ""

for text in chunks:
    if summary == "":
        input = text
    else:
        input = summary + "\n" + text

    # 先summary
    summary, history  = model.chat(tokenizer, summary_prompt + input, history=[])

    input_text = f"paragraph:\n{input}\n\nsummary:\n{summary}"

    csv_response, history = model.chat(tokenizer, input_text, system = extract_prompt, history=[])

    print(summary)
    print(csv_response)
    print('-----')

    # 加个解析再加个清洁（删掉recall < 20的句子)

    continue



故事发生在一个清晨，主人公老刀发现彭蠡不在家，猜测他在外面匆忙回家后吃了早饭。在这个狭小的小区里，人们忙碌地为生活奔波。主人公看到其他人买昂贵的食物，心里产生不安和饥饿感。他思考自己的处境，决定寻找机会摆脱困境。整个场景描绘了城市生活的喧嚣和人们的忙碌，以及主人公内心的矛盾与挣扎。
dialogue text | said by | speaker
"哎，你们知道那儿一盘回锅肉多少钱吗？" | said by | 小李
"靠，菜里有沙子。" | said by | 小丁
"三百四！一盘水煮牛肉四百二呢。" | said by | 小李
"什么玩意？这么贵。" | said by | 小丁
"啊……我来这一行也二十几年了，从来没有赚那么多钱过。那小子呢，连我自己也不管了，明里暗里欺负女人……而且我都是站着的。一个月下来，腿上的皮都磨破了。" | said by | 小张
-----
故事发生在清晨，主人公老刀发现彭蠡不在家，猜测他在外面匆忙回家后吃了早饭。在狭小的小区里，人们忙碌地为生活奔波。主人公看到其他人买昂贵的食物，心里产生不安和饥饿感。他思考自己的处境，决定寻找机会摆脱困境。

此时，彭蠡出现了。他是一个懒散、不修边幅的人，但是年轻时很有作为。彭蠡看到老刀要离开，感到有些惊讶。老刀告诉他要去的第一空间，彭蠡犹豫了一下，但最终还是让他跟着走。

在路上，他们经过了一个废弃的仓库区，彭蠡直接把衣服扔进了旁边的河里。老刀问为什么这样做，彭蠡说是处理一些琐事。老刀简单地讲述了自己如何成为黑社会老大，最后如何被人追杀的故事。彭蠡表示愿意支付赎金来解决这件事，并表示想要让老刀消失一段时间。

老刀告诉彭蠡他的计划需要等待二十四小时才能执行。彭蠡表示同意，并询问老刀还需要什么帮助。老刀告诉彭蠡他需要一百万人民币来购买毒品以避免被抓住。彭蠡表示理解并承诺支持他完成这个任务。
dialogue text | said by | speaker
"我没时间和你解释。我需要去第一空间，你告诉我怎么走。" | said by | 老刀
"回我家说，要走也从那儿走。" | said by | 彭蠡
"在家门口坐等吧，回去一看都五点半了，你少在那里晃悠。" | said by | 彭蠡
"你不告诉我为什么，我就不告诉你怎么走。" | said by | 彭蠡
"你躲在垃圾道里？去第二空间

In [24]:
#@title define raw_text

raw_text = """ I will always remember the view of Paris from his window. Snow, pure and untouched, softens the outline of the buildings and covers the grime of the streets. White, the color of beginnings. His canvas is primed and ready to be painted, and stark winter sunlight glows bright on his undead skin.

The studio is cramped, drafty despite the heat radiating from the stove. One corner is clean and lavishly decorated, the rest a cluttered chaos of painting supplies and personal effects. He studies me intently as I take in the room, evaluating me much as he did at the Café Guerbois when I’d first caught his eye.

I wait for him to ask how I came to be in Paris. Artists are so very predictable that way—no trouble at all accepting this pale immortal creature as one of their own, but a woman of my mixed ancestry? Utterly implausible.

“You should hear the stories they tell of you at the café,” he says. “If Émile is to be believed, you arrived here as an ukiyo-e courtesan, nothing more than paper wrapped around a porcelain bowl. A painter—he will not say which of us it was, of course—bought the bowl and the print along with it.”

“And the painter pulled me from the print with the sheer force of his imagination, I’m sure,” I reply, laughing. “Émile is a novelist and can hardly be trusted to give an accurate account. The reality of my conception is vastly more mundane, I assure you…though it does involve a courtesan.”

“A grain of truth makes for the best fiction.” He waves his hand at a worn-looking dressing screen. “Nude, but leave the jewelry and the shoes. I’ll paint you on the chaise. We’ll have three hours in the proper light, and I will pay you four francs.”

“Victorine gets five!” I protest from behind the screen as I get undressed.

“Victorine is a redhead.”

I step out from behind the screen and go to the chaise, running my fingers along the elegant curves of the walnut frame. The cushions are firm and covered in soft green velvet. I arrange myself carefully. Hopefully he will like what he sees. Often what the artists demand is a relaxed-looking pose that is hideously uncomfortable. Like novelists, they require only a grain of truth. The rest is purely of their own creation.

“My name is Mariko, by the way, but everyone calls me Mari.” As if I could pass for a French girl simply by changing my name. Though, particularly with the artists, there is a fascination with all things Japanese. Several of Hokusai’s views of Mount Fuji decorate the wall behind me, the ukiyo-e prints crammed together with neoclassical portraits and a few realist landscapes of the Barbizon School.

He remains facing the window, his attention fixed on the snowy landscape.

“I’m on the chaise,” I tell him, and finally he turns.

“Bring your left hip forward. No, not that far. Bend the leg a bit more, yes.” He paces back and forth, frowning. “Turn your head to face the canvas.”

I smile knowingly. “Like a Manet.”

His frown deepens into a scowl.

“Don’t like a model that talks while you work, huh?” I’ve posed for that type before, honestly not my favorite sort of job, there to be seen and not heard. If the artist is talented enough I can still pick up a technique or two watching them work, but—

“I don’t like being compared to other artists.”

I laugh. More of an ego than usual, this one. Though perhaps he’s earned it. If Victorine was to be believed, he’s been painting since the Renaissance. “Then you must paint me so well that I forget about the others.”

“Tilt your head into the light.” His voice is softer now, and he steps forward to cup my chin, shifting the angle of my head ever so slightly to refine the pose. “And look at me intently. Intensely. As though I were the one naked on the chaise.”

His touch sends shivers down my spine. It feels as if he is reaching into me, beyond the surface of my skin. Intimate. I’m not above a dalliance with an artist if he pleases my eye, or if I need the money or a place to stay…but this one is different.

His eyes are as dark as the Seine at night, darker even than my own. I’m laid bare before him in more ways than my mere lack of clothing. The canvas is reflected in the window behind him, and he is painting me in deft strokes of vivid color—as other artists have done before him—but this time the image holds the promise of an understanding. His skill with the paint is breathtaking; his movements simultaneously wild and precise.

It is exhilarating to watch him work.

My back aches and one leg is going numb, but I’m disappointed when he sets down his brush.

“You did better than I would have expected.”

“Oh?” I stretch and, still nude, go to take a closer look at the canvas. Even with the work unfinished, I can see that he is more talented than any of the other artists I’ve known, and his intensity sparks my interest, draws me almost inevitably closer. “There are other poses I could show you, if you like?”


“Hmmm…?” His gaze is fixed on the canvas, studying a streak of bright winter sunlight that cuts across the upper corner.

I’m about to give him up as hopeless when he turns to look at me. I’m lost in the darkness of his eyes, drowning in the intensity of his attention. I can barely breathe, but I repeat my invitation, “I could show you other poses.”

“Yes.” He sweeps me into an embrace that is strong and cold. White. He is snow and I am determined to melt it.

The sex builds slowly, deliberately, like paint layered on a canvas in broad strokes—tentative at first as we find our way to a shared vision, then faster with a furious intensity and passion.

After, when other artists might hold me and drift off to sleep, he dissipates into a white mist that swirls in restless circles around the room, chilling me down to the bones when it touches my skin. His mist seeps into me and pulses through my veins for several heartbeats. I feel energized, an exhilaration more intense than watching him work, a connection closer even than our sex.

He withdraws, and I am diminished. I hadn’t known until this moment what I was lacking, but now I am filled with a keen sense of my incompleteness. I long for him, for the sensation of vastness I felt when we were one.

He does not return to the bed.

I sleep alone and wake to windows white with frost.
"""

In [25]:
TOKEN_PER_CHUNK = 600

lines = raw_text.split("\n")

chunks = []
current_chunk = ""
current_len = 0

for line in lines:
    if line.strip() == "":
        continue
    line_len = get_token_len(line)
    if current_len + line_len > TOKEN_PER_CHUNK:
        if current_len > 0:
            chunks.append(current_chunk)
            current_chunk = line
            current_len = line_len

    else:
        current_chunk += line+ "\n"
        current_len += line_len

if current_len > 0:
    chunks.append(current_chunk)

print(len(chunks), " chunks")

3  chunks


In [26]:
summary_prompt = """Summarize the input data and output directly."""
extract_prompt = """Given an input paragraph and the corresponding summary, extract the text dialogue, and determine the speaker of each. Output in CSV format in the form of <dialogue text> | said by | <speaker>."""

In [27]:
import json
summary = ""
model = model.eval()

dsp_text = ""

for text in chunks:
    if summary == "":
        input = text
    else:
        input = summary + "\n" + text

    # 先summary
    summary, history  = model.chat(tokenizer, summary_prompt + input, history=[])

    input_text = f"paragraph:\n{input}\n\nsummary:\n{summary}"

    csv_response, history = model.chat(tokenizer, input_text, system = extract_prompt, history=[])

    print(summary)
    print(csv_response)
    print('-----')

    # 加个解析再加个清洁（删掉recall < 20的句子)

    continue



A woman is being interviewed about her arrival in Paris and her past as an ukiyo-e courtesan. She is asked about the painter who sold her a print and how she ended up in Paris. They agree on a price of four francs and she prepares herself for a portrait session. She introduces herself as Mariko and explains her changed name due to the influence of Japanese art.
dialogue text | said by | speaker
"You should hear the stories they tell of you at the café." | said by | Artist
"If Émile is to be believed, you arrived here as an ukiyo-e courtesan, nothing more than paper wrapped around a porcelain bowl. A painter—he will not say which of us it was, of course—" | said by | Artist
"And the painter pulled me from the print with the sheer force of his imagination, I’m sure." | said by | Woman
"Nude, but leave the jewelry and the shoes. I’ll paint you on the chaise. We’ll have three hours in the proper light, and I will pay you four francs." | said by | Artist
"Victorine gets five!" | said by | W